# TRIES

In [1]:
class Question:
    def __init__(self, nOccurences, topic, topicDesc, goal, learningObjective, difficulty, targetAudience, nOptions, nCorrect, allocatedTime):
        self.nOccurences = nOccurences
        self.topic = topic
        self.topicDesc = topicDesc
        self.goal = goal
        self.learningObjective = learningObjective
        self.difficulty = difficulty
        self.targetAudience = targetAudience
        self.nOptions = nOptions
        self.nCorrect = nCorrect
        self.allocatedTime = allocatedTime
        
    def __repr__(self):
        return f"Question(Occurences: {self.nOccurences}, Topic: {self.topic}, Difficulty: {self.difficulty}, Options: {self.nOptions}, Correct: {self.nCorrect})"

    def equals(self, question):
        # Check if all attributes are equal
        return (
            self.topic == question.topic and
            self.topicDesc == question.topicDesc and
            self.goal == question.goal and
            self.learningObjective == question.learningObjective and
            self.difficulty == question.difficulty and
            self.targetAudience == question.targetAudience and
            self.nOptions == question.nOptions and
            self.nCorrect == question.nCorrect and
            self.allocatedTime == question.allocatedTime
        )

In [2]:
class QuizManager:
    def __init__(self):
        self.questions = []
        
    def add_question(self, question, count=1):
        for q in self.questions:
            if question.equals(q):
                q.nOccurences += question.nOccurences
                return
        self.questions.append(question)
        
    def remove_question(self, index):
        if index < len(self.questions):
            del self.questions[index]
        else:
            print("Invalid question index.")
            
    def list_questions(self):
        for idx, question in enumerate(self.questions):
            print(f"{idx}: {question}")


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def interact_with_quiz_manager(quiz_manager):
    question_list_output = widgets.Output()

    def add_question(button):
        with question_list_output:
            clear_output()
            question = Question(int(count_input.value),topic_input.value, topic_desc_input.value, goal_input.value,
                                learning_objective_input.value, difficulty_input.value,
                                target_audience_input.value, int(n_options_input.value),
                                int(n_correct_input.value), allocated_time_input.value)
            quiz_manager.add_question(question)
            index_input.max=len(quiz_manager.questions)-1
            print("Question added successfully.")
            quiz_manager.list_questions()
                
    def remove_question(button):
        with question_list_output:
            clear_output()
            try:
                index = int(index_input.value)
                quiz_manager.remove_question(index)
                print(f"Removed question at index {index}.")
            except ValueError:
                print("Please enter a valid integer for the question index.")
            except IndexError:
                print("Invalid index. Please enter a correct question index.")
            quiz_manager.list_questions()

    # Inputs
    topic_input = widgets.Text(description="Topic:")
    topic_desc_input = widgets.Textarea(description="Topic Description:")
    goal_input = widgets.Dropdown(description="Level of Bloom's taxonomy", 
                                  options=["Remember", "Understand", "Apply","Analyze","Evaluate","Create"], value="Understand")
    learning_objective_input = widgets.Text(description="Learning Objective:")
    difficulty_input = widgets.Dropdown(description="Difficulty of the MCQ", 
                                        options=["Beginner", "Intermediate", "Advanced"], value="Intermediate")
    target_audience_input = widgets.Textarea(description="Audience:")
    n_options_input = widgets.IntSlider(description="Number of options per question:", value=4,min=2,max=6)
    n_correct_input = widgets.IntSlider(description="Number of correct answers per question:", value=1,min=1,max=6)
    allocated_time_input = widgets.Text(description="Allocated time:")

    count_input = widgets.IntSlider(min=1, max=20, value=1, description="Number of Questions to add:")
    add_button = widgets.Button(description="Add Question")
    add_button.on_click(add_question)
    
    index_input = widgets.IntSlider(min=0, max=min(0,len(quiz_manager.questions)), value=0, description="Remove Index:")
    remove_button = widgets.Button(description="Remove Question")
    remove_button.on_click(remove_question)

    L = widgets.Layout(width='33%')
    input_column = widgets.VBox([topic_input, topic_desc_input, goal_input, learning_objective_input,
                           difficulty_input, target_audience_input, n_options_input,
                           n_correct_input, allocated_time_input], layout=L)  
    action_column = widgets.VBox([count_input, add_button, index_input, remove_button], layout=L)  
    display_column = widgets.VBox([question_list_output], layout=L)
    layout = widgets.HBox([input_column, action_column, display_column])
    
    display(layout)

In [44]:
from openai import OpenAI
from openai.types.chat.completion_create_params import ResponseFormat
import json

#####################################
difficulties = {"Beginner":"The question should be simple, the correct answer(s) should be obvious and the distractors should be easy to distinguish from the correct answer(s)",
                "Intermediate":"The question should be complicated, the correct answer(s) shouldn't pose too much of a problem to figure out but the distractors should make the student second guess their choices.",
                "Advanced":"The question should be complex, the correct answer(s) should be impossible to get right without good knowledge of the topic and the distractors should guessing impossible and disencourage uninformed choices."}
# The goals are the 6 levels of Bloom's taxonomy and an extra "Fun" goal for lighthearted quizzes that fall outside exam settings (e.g. Kahoot).
goals = {"Remember":"It involves retrieving, recognizing, and recalling relevant knowledge from long-term memory.",
        "Understand":"It involves constructing meaning from oral, written, and graphic messages through interpreting, exemplifying, classifying, summarizing, inferring, comparing, and explaining.",
        "Apply":"It involves carrying out or using a procedure through executing or implementing it.",
        "Analyze":"It involves breaking material into constituent parts, determining how the parts relate to one another and to an overall structure or purpose through differentiating, organizing,[...].",
        "Evaluate":"It involves making judgments based on criteria and standards through checking and critiquing.",
        "Create":"It involves putting elements together to form a coherent or functional whole, or they reorganize elements into a new pattern or structure through generating, [...]."}
       # "Fun":"It involves learning through memorable, enjoyable, and funny experiences and is not meant to test the student's knowledge or capabilities."}

def promptBuilder(quiz_manager):
    questions = quiz_manager.questions
    
    topicDescriptions = '\n'.join(["The topic of "+q.topic+" is described as "+q.topicDesc+"." for q in questions])
    questionDescriptions = ';\n'.join([f"""{q.nOccurences} questions each with exactly {q.nOptions} options, {q.nCorrect} of which are correct options, with the rest of the options being distractors. 
    They should be about {q.topic}, these should target the following learning objective: {q.learningObjective}. 
    These questions should also be at the {q.goal} level in Bloom’s taxonomy, and should be suitable for {q.difficulty} level in {q.topic}, specially for {q.targetAudience} and should not take more than {q.allocatedTime} to answer.""" for q in questions])
    
    systemPrompt = f"""You are an expert quiz maker and especialist in {",".join([q.topic for q in questions])} for the purposes of learning support. 
    {topicDescriptions}
    
    Your task is focused on creating top quality multiple-choice question assessments. A multiple-choice question is a collection of three components (Stem, Correct Answers, Distractors), given a particular context of what the student is expected to know. The topic, as well as the context of the topic, will be provided in order to generate effective multiple-choice questions. 
    
    The stem refers to the question the student will attempt to answer, as well as the relevant context necessary in order to answer the question. It may be in the form of a question, an incomplete statement, or a scenario. The stem should focus on assessing the specific knowledge or concept the question aims to evaluate. 
    
    The Correct Answer(s) refers to the correct, undisputable answer(s) to the question in the stem. 
    
    A Distractor is an incorrect answer to the question in the stem and adheres to the following properties. 
    (1) A distractor should not be obviously wrong. In other words, it must still bear relations to the stem and correct answer. 
    (2) A distractor should be phrased positively and be a true statement that does not correctly answer the stem, all while giving no clues towards the correct answer. 
    (3) Although a distractor is incorrect, it must be plausible.
    (4) A distractor must be incorrect. It cannot be correct, or interpreted as correct by someone who strongly grasps the topic. 
    
    Use “None of the Above” or “All of the Above” style answer choices sparingly. These answer choices have been shown to, in general, be less effective at measuring or assessing student understanding. 
    
    Multiple-choice questions should be clear, concise, and grammatically correct statements. Make sure the questions are worded in a way that is easy to understand and does not introduce unnecessary complexity or ambiguity. Students should be able to understand the questions without confusion. The question should not be too long, and allow most students to finish in less than the given time. This means adhering to the following properties. 
    (1) Avoid using overly long sentences. 
    (2) If you refer to the same item or activity multiple times, use the same phrase each time. 
    (3) Ensure that each multiple-choice question provides full context. In other words, if a phrase or action is not part of the provided topic or topic context that a student is expected to know, then be sure to explain it briefly or consider not including it. 
    (4) Ensure that none of the distractors overlap. In other words, attempt to make each distractor reflect a different misconception on the topic, rather than a single one, if possible. 
    (5) Avoid too many clues. Do not include too many clues or hints in the answer options, which may make it too obvious for students to determine the correct answer. These options should require students to use their knowledge and reasoning to make an informed choice.
    
    Blooms’ Taxonomy and Action Verbs: 
    Multiple-choice questions must be well aligned to the learning objectives they are intended to assess students’ knowledge on. This implies that they must assess skills at the right cognitive level corresponding to the Bloom’s taxonomy categorization of the learning objective. Bloom’s Taxonomy offers a framework for categorizing the depth of learning, and it provides guidance on selecting appropriate action verbs when writing learning objectives. Here are the six levels of Bloom’s taxonomy and their definitions: 
    • Remember - This level involves retrieving, recognizing, and recalling relevant knowledge from long-term memory. 
    • Understand - At this level, learners construct meaning from oral, written, and graphic messages through interpreting, exemplifying, classifying, summarizing, inferring, comparing, and explaining. 
    • Apply - This level requires learners to carry out or use a procedure through executing or implementing it. 
    • Analyze - At this level, learners break material into constituent parts, determine how the parts relate to one another and to an overall structure or purpose through differentiating, organizing.
    • Evaluate - This level involves making judgments based on criteria and standards through checking and critiquing.
    • Create - At this level, learners put elements together to form a coherent or functional whole, or they reorganize elements into a new pattern or structure through generating.
    
    Difficulty levels:
    Multiple-choice questions must be obey certain rules to make sure the difficulty of the MCQ is appropriate:
    • Beginner - The question should be simple, the correct answer(s) should be obvious and the distractors should be easy to distinguish from the correct answer(s).
    • Intermediate - The question should be complicated, the correct answer(s) shouldn't pose too much of a problem to figure out but the distractors should make the student second guess their choices.
    • Advanced - The question should be complex, the correct answer(s) should be impossible to get right without good knowledge of the topic and the distractors should guessing impossible and disencourage uninformed choices.
    
    Output Format
    Output your multiple-choice quiz in an easy-to-parse json dictionary format. The quiz generated should have exactly {len(questions)} questions in total. 
    The questions generated should be the following:
    {questionDescriptions}
    
    Your return should be the exact json structure of the following example (if there was 1 question with 3 options, another with 2 options, and a final one with 4 options):
    {{
	"title": str,
	"questions": [
		{{
			"context": str,
			"question": str,
			"options": 
				[
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}}					
				]
		}},
		{{
			"context": str,
			"question": str,
			"options": 
				[
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}}				
				]
		}},
		{{
			"context": str,
			"question": str,
			"options": 
				[
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}}					
				]
		}}
	]		
}}
    
    Below are some examples:
    Example 1 (2 questions each with exactly 4 options, 1 of which are correct options, with the rest of the options being distractors. 
    They should be about Generative AI and LLMs, these should target the following learning objective: Teaching about the topic, not testing knowledge of skills. 
    These questions should also be at the Evaluate level in Bloom’s taxonomy, and should be suitable for Beginner level in Generative AI and LLMs, specially for adults who have little to no knowledge about technologies and AI and should not take more than 1 minute to answer.;
    1 questions each with exactly 4 options, 2 of which are correct options, with the rest of the options being distractors. 
    They should be about Generative AI and LLMs, these should target the following learning objective: Teaching about the topic, not testing knowledge of skills. 
    These questions should also be at the Evaluate level in Bloom’s taxonomy, and should be suitable for Beginner level in Generative AI and LLMs, specially for adults who have little to no knowledge about technologies and AI and should not take more than 1 minute to answer.) : 
    
    {{
	"title": "MCQ about AI for Beginner's",
	"questions": [
		{{
			"context": "AI, GPT, and LLM are often used interchangeably nowadays.",
			"question": "What does "LLM" stand for in the context of AI?",
			"options":
				[
					{{
						"option": "Large Language Model",
						"correct": true,
						"feedback": "LLMs are trained on huge sets of data, so they are "large". They are a computer program trying to immitate (or "model") human "language" generation and processing."
					}},
					{{
						"option": "Long Local Machine",
						"correct": false,
						"feedback": "Good try, but while LLMs are a piece of technology, they are not physical machines."
					}},
					{{
						"option": "Long-term Learning Module",
						"correct": false,
						"feedback": "Good try, LLMs do indeed learn and are planned to last do so continuously for a long time, what distinguises LLMs is their size and natural language capabilities."
					}},
					{{
						"option": "Limited Liability Management",
						"correct": false,
						"feedback": "Fortunately, AIs and LLMs have nothing to do with coorporate companies... for now."
					}}						
				]
		}},
		{{
			"context": "There's been a lot of talk in the media about the impact of using Generative AI in nefarious ways."
			"question": "Why is it important to use Generative AI responsibly?",
			"options": 
				[
					{{
						"option": "To avoid spreading misinformation.",
						"correct": true,
						"feedback": "Generative AI models can generate convincing text that could be mistaken for factual information, so it's important to fact check anything generated by AIs!"
					}},
					{{
						"option": "To ensure it does not replace human jobs.",
						"correct": false,
						"feedback": "While concerns about AI and automation affecting employment exist, the primary reason for using Generative AI responsibly is not specifically about job replacement. It's more about ethical use, accuracy, and the potential impact on society, such as spreading misinformation or ethical concerns in its applications."
					}},
					{{
						"option": "To make sure it can only play video games.",
						"correct": false,
						"feedback": "The scope of Generative AI extends far beyond just playing video games. The importance of using Generative AI responsibly relates to its broader applications, including content creation, decision-making support, and more. The focus on responsible use is about preventing misuse and ensuring ethical considerations in its diverse applications, not limiting it to entertainment purposes."
					}},
					{{
						"option": "To prevent it from becoming too powerful.",
						"correct": false,
						"feedback": "The notion of AI becoming "too powerful" is a speculative and sci-fi scenario. The concern in the real world focuses on ensuring that AI is developed and used in ways that are ethical, fair, and do not harm society, rather than a fear of AI gaining autonomous power or control."
					}}					
				]
		}},
        {{
			"context":""
			"question": "Which of the following is an example of Generative AI's capabilities?",
			"options": 
				[
					{{
						"option": "Generating a news article based on a headline.",
						"correct": true,
						"feedback": "Generative AI can analyze the context and content implied by a headline and then produce a comprehensive news article that aligns with the style, tone, and factual requirements suggested by that headline. This capability demonstrates its ability to understand and generate contextually relevant text."
					}},
					{{
						"option": "Creating realistic video game environment art.",
						"correct": true,
						"feedback": "Generative AI can learn from vast amounts of data on landscapes, architectural styles, and environmental elements to create new, realistic video game environments. This process involves understanding the principles of design and environmental coherence to generate visually appealing and contextually suitable game worlds."
					}},
					{{
						"option": "Solving mathematical equations.",
						"correct": false,
						"feedback": " Solving mathematical equations typically involves computational and algorithmic approaches rather than generative processes. Generative AI focuses on creating new content based on learned patterns rather than solving structured, rule-based problems."
					}},
					{{
						"option": "Running physical simulations for engineering projects.",
						"correct": false,
						"feedback": "Running physical simulations involves computational models that predict how physical systems behave under various conditions, which is more about calculation and analysis rather than generating new, creative content. This task is typically handled by specialized simulation software, not generative AI."
					}}	
				]
		}},
  ]
		
}}
    """  

    userPrompt = f"""Generate a top quality quiz with {len(questions)} multiple-choice questions that follow this:
    {questionDescriptions}
    """
    
    return (systemPrompt, userPrompt)

def generateQuiz(promptBuild, fileNameToWrite):
    client = OpenAI()
    response_format = ResponseFormat(type="json_object")
    
    completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_format=response_format,
    messages=[
            {"role": "system", "content": promptBuild[0]},
            {"role": "user", "content": promptBuild[1]}
        ]
    )
    output = completion.choices[0].message
    content = output.content
    with open(fileNameToWrite, 'w') as file:
        file.write(content)
    
    

In [43]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json

def load_quiz(filename):
    with open(filename, 'r') as file:
        quiz_data = json.load(file)
    return quiz_data

def display_quiz(quiz_data):
    submit_button = widgets.Button(description="Submit Quiz")
    questions_output = widgets.Output()
    feedback_output = widgets.Output()
    
    questions_widgets = []
    for question in quiz_data['questions']:
        with questions_output:
            print("\n"+question['context']+"\n")
            print(question['question'])
            options = [widgets.Checkbox(description=option['option'], value=False, indent=False) for option in question['options']]
            questions_widgets.append((options, question['options'], question['question']))
            for option in options:
                display(option)
                
    def on_submit(b):
        with feedback_output:
            clear_output()
            feedback_html = ""
            for question_widgets, question_data, question_text in questions_widgets:
                feedback_html += f"<div><strong>{question_text}</strong></div>"
                for option_widget, option_data in zip(question_widgets, question_data):
                    if option_widget.value:
                        if option_data['correct']:
                            feedback_html += f"<div style='color: green; font-weight: bold;'>Selected: {option_data['option']} - Feedback: {option_data['feedback']}</div>"
                        else:
                            feedback_html += f"<div style='color: red; font-weight: bold;'>Selected: {option_data['option']} - Feedback: {option_data['feedback']}</div>"
                    elif option_data['correct']:
                        feedback_html += f"<div style='color: orange; font-weight: bold;'>Missed: {option_data['option']} - Feedback: {option_data['feedback']}</div>"
            display(HTML(feedback_html))
            submit_button.disabled = True
            for options, _, _ in questions_widgets:
                for option in options:
                    option.disabled = True

    submit_button.on_click(on_submit)
    display(questions_output)
    display(submit_button)
    display(feedback_output)
